# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |Damarys Valenzuela Santos   |
| **Fecha**      |  09/09/2025 |
| **Expediente** | 744494  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente.

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [6]:
df = pd.read_csv("Advertising.csv")
X = df[["TV", "radio", "newspaper"]].values
y = df["sales"].values

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.4,random_state=137)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

# Regresión sin penalización

In [12]:
modelo = LinearRegression()
modelo.fit(Xtrain_scaled, ytrain)
y_train_pred = modelo.predict(Xtrain_scaled)
y_test_pred = modelo.predict(Xtest_scaled)

In [13]:
r2_score(ytest, y_test_pred), r2_score(ytrain, y_train_pred)

(0.8655966025115787, 0.926929129172944)

In [14]:
modelo.coef_,modelo.intercept_

(array([4.15454877, 2.91555436, 0.03329485]), np.float64(13.89375))

In [15]:
X_with_const = sm.add_constant(Xtrain_scaled)
modelo_sm = sm.OLS(ytrain, X_with_const).fit()

print(modelo_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     321.4
Date:                Wed, 10 Sep 2025   Prob (F-statistic):           4.50e-43
Time:                        15:34:18   Log-Likelihood:                -149.41
No. Observations:                  80   AIC:                             306.8
Df Residuals:                      76   BIC:                             316.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.8937      0.180     77.331      0.0

# L2

In [16]:
ridge = Ridge(alpha=1.0)
ridge.fit(Xtrain_scaled, ytrain)
y_train_ridge = ridge.predict(Xtrain_scaled)
y_test_ridge = ridge.predict(Xtest_scaled)

In [17]:
ridge.coef_,ridge.intercept_

(array([4.10467047, 2.881016  , 0.05783146]), np.float64(13.89375))

In [18]:
r2_score(ytest, y_test_ridge), r2_score(ytrain, y_train_ridge)

(0.8654818952380672, 0.9268163840897949)

In [31]:
n = Xtrain_scaled.shape[0]
p = Xtrain_scaled.shape[1]
resid = ytrain - y_train_ridge
RSS = np.sum(resid**2)
sigma2 = RSS / (n - p - 1)
X_with_intercept = np.hstack([np.ones((n,1)), Xtrain_scaled])
cov_matrix = sigma2 * np.linalg.inv(X_with_intercept.T @ X_with_intercept)
SEs = np.sqrt(np.diag(cov_matrix))
coefs = np.hstack([ridge.intercept_, ridge.coef_])
t_stats = coefs / SEs
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p-1))
for i, name in enumerate(["b0 (intercept)", "b1 TV", "b2 Radio", "b3 Newspaper"]):
    print(f"{name} = {coefs[i]:.4f}, t = {t_stats[i]:.4f}, p-value = {p_values[i]:.4f}")

b0 (intercept) = 13.8938, t = 77.2717, p-value = 0.0000
b1 TV = 4.1047, t = 21.9253, p-value = 0.0000
b2 Radio = 2.8810, t = 14.6965, p-value = 0.0000
b3 Newspaper = 0.0578, t = 0.2929, p-value = 0.7704


# L1

In [37]:
lasso = Lasso(alpha=0.1)
lasso.fit(Xtrain_scaled, ytrain)
y_train_lasso = lasso.predict(Xtrain_scaled)
y_test_lasso = lasso.predict(Xtest_scaled)

In [38]:
lasso.coef_,lasso.intercept_

(array([4.07798137, 2.84462045, 0.        ]), np.float64(13.89375))

In [39]:
r2_score(ytest, y_test_lasso), r2_score(ytrain, y_train_lasso)

(0.8663284653669636, 0.9264116155676066)

In [40]:
n = Xtrain_scaled.shape[0]
p = Xtrain_scaled.shape[1]
resid = ytrain - y_train_lasso
RSS = np.sum(resid**2)
sigma2 = RSS / (n - p - 1)
X_with_intercept = np.hstack([np.ones((n,1)), Xtrain_scaled])
cov_matrix = sigma2 * np.linalg.inv(X_with_intercept.T @ X_with_intercept)
SEs = np.sqrt(np.diag(cov_matrix))
coefs = np.hstack([lasso.intercept_, lasso.coef_])
t_stats = coefs / SEs
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p-1))
for i, name in enumerate(["b0 (intercept)", "b1 TV", "b2 Radio", "b3 Newspaper"]):
    print(f"{name} = {coefs[i]:.4f}, t = {t_stats[i]:.4f}, p-value = {p_values[i]:.4f}")

b0 (intercept) = 13.8938, t = 77.0589, p-value = 0.0000
b1 TV = 4.0780, t = 21.7227, p-value = 0.0000
b2 Radio = 2.8446, t = 14.4709, p-value = 0.0000
b3 Newspaper = 0.0000, t = 0.0000, p-value = 1.0000


Al comparar los modelos Lineal, Ridge y Lasso, todos obtuvieron resultados muy similares, con R² alrededor de 0.92 en train y 0.85 en test. Esto indica que los tres modelos ajustan bien los datos y no hay sobreajuste importante. Los coeficientes y p-values de LinearRegression también fueron similares a los de Ridge y Lasso, lo que muestra que la penalización ligera no cambió mucho los resultados. En general, las variables TV y radio son significativas mientras que periodico no.
